In [8]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import time

In [9]:
# Create a new set of 2,000 random latitudes and longitudes.
lats = np.random.uniform(-90.000, 90.000, size=2000)
lngs = np.random.uniform(-180.000, 180.000, size=2000)
lats_lngs = zip(lats, lngs)
lats_lngs

In [10]:
# Add the latitudes and longitudes to a list.
coordinates = list(lats_lngs)
coordinates

[(-60.128088316275495, -42.95942695955824),
 (-3.336019382109427, 1.4019826983121106),
 (12.77371771293383, -66.59248571375247),
 (-58.25799029848692, -139.7535306319439),
 (-14.846082183020243, 121.05651345556305),
 (2.977797899426264, -6.003845770565306),
 (-34.16763481221123, -39.60353652599812),
 (-4.683102081237365, 12.639345716907684),
 (-31.020275457884466, 136.27316527124248),
 (31.613318556040653, 47.279457549235076),
 (44.741640380818865, 120.4730837494568),
 (-0.13162842579386336, -79.17562712867917),
 (-23.56511024790791, -95.4873818859241),
 (49.058573599797256, 76.95925255187285),
 (66.47969758557764, 137.27939702629158),
 (-29.09457312940149, -56.48478386587634),
 (41.42450639299386, -79.53296830814311),
 (-34.12853383623895, 131.85505970649496),
 (59.44289916431964, 117.3528341374294),
 (48.02652567463744, 160.68823665199216),
 (-8.565920456014538, -150.32644060171378),
 (65.52032886670398, -142.56881938173223),
 (-76.39848842659127, -162.16573564104453),
 (9.1530802710

In [11]:
from citipy import citipy
import pandas as pd
from keys import weather_api_key
from config import g_key


In [12]:
# Get the nearest city using the citipy module.
cities = list()
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name
    if city not in cities:
        cities.append(city)
print(cities[:10], 'Generate', len(cities))

['ushuaia', 'port-gentil', 'kralendijk', 'rikitea', 'broome', 'san-pedro', 'laguna', 'tshela', 'port augusta', 'susangerd'] Generate 754


In [17]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key


In [18]:
# List of city data
city_data = []
# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")
# Create counters
record_count = 1
set_count = 1
# Loop through all the cities in our list
for i, city in enumerate(cities):
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city
    city_url = url + "&q=" + city.replace(" ","+")
    # Log the url, record, and set numbers
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count
    record_count += 1
    # Run an API request for each of the cities
    try:
        city_weather = requests.get(city_url).json()
        # Parse out the max temp, humidity, and cloudiness
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather["weather"][0]["description"]
            # Append the City information into city_data list
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Current Description": city_weather_description,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country})
    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass
# Indicate that Data Loading is complete
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | ushuaia
Processing Record 2 of Set 1 | port-gentil
Processing Record 3 of Set 1 | kralendijk
Processing Record 4 of Set 1 | rikitea
Processing Record 5 of Set 1 | broome
Processing Record 6 of Set 1 | san-pedro
Processing Record 7 of Set 1 | laguna
Processing Record 8 of Set 1 | tshela
Processing Record 9 of Set 1 | port augusta
Processing Record 10 of Set 1 | susangerd
Processing Record 11 of Set 1 | wulanhaote
City not found. Skipping...
Processing Record 12 of Set 1 | santo domingo
Processing Record 13 of Set 1 | pisco
Processing Record 14 of Set 1 | karkaralinsk
City not found. Skipping...
Processing Record 15 of Set 1 | batagay
Processing Record 16 of Set 1 | itaqui
Processing Record 17 of Set 1 | oil city
Processing Record 18 of Set 1 | flinders
Processing Record 19 of Set 1 | kropotkin
Processing Record 20 of Set 1 | severo-kurilsk
Processing Record 21 of Set 1 | faanui
Processing Record 2

Processing Record 36 of Set 4 | urumqi
Processing Record 37 of Set 4 | goundam
Processing Record 38 of Set 4 | hebertville
City not found. Skipping...
Processing Record 39 of Set 4 | whitehorse
Processing Record 40 of Set 4 | san patricio
Processing Record 41 of Set 4 | sao joao da barra
Processing Record 42 of Set 4 | saldanha
Processing Record 43 of Set 4 | qaqortoq
Processing Record 44 of Set 4 | ugoofaaru
Processing Record 45 of Set 4 | muswellbrook
Processing Record 46 of Set 4 | zhanatas
City not found. Skipping...
Processing Record 47 of Set 4 | karratha
Processing Record 48 of Set 4 | mpanda
Processing Record 49 of Set 4 | kuche
City not found. Skipping...
Processing Record 50 of Set 4 | tual
Processing Record 1 of Set 5 | sitka
Processing Record 2 of Set 5 | kautokeino
Processing Record 3 of Set 5 | marawi
Processing Record 4 of Set 5 | amderma
City not found. Skipping...
Processing Record 5 of Set 5 | coihaique
Processing Record 6 of Set 5 | srednekolymsk
Processing Record 7 

Processing Record 22 of Set 8 | opuwo
Processing Record 23 of Set 8 | labuhan
Processing Record 24 of Set 8 | pangnirtung
Processing Record 25 of Set 8 | zhigansk
Processing Record 26 of Set 8 | borogontsy
Processing Record 27 of Set 8 | muzhi
Processing Record 28 of Set 8 | obidos
Processing Record 29 of Set 8 | ormara
Processing Record 30 of Set 8 | vao
Processing Record 31 of Set 8 | mazagao
Processing Record 32 of Set 8 | diban
Processing Record 33 of Set 8 | hami
Processing Record 34 of Set 8 | sankheda
Processing Record 35 of Set 8 | darhan
Processing Record 36 of Set 8 | mayo
Processing Record 37 of Set 8 | fort nelson
Processing Record 38 of Set 8 | montreal
Processing Record 39 of Set 8 | ekibastuz
Processing Record 40 of Set 8 | kushiro
Processing Record 41 of Set 8 | hay river
Processing Record 42 of Set 8 | bambari
Processing Record 43 of Set 8 | mporokoso
Processing Record 44 of Set 8 | bacolod
Processing Record 45 of Set 8 | saint-joseph
Processing Record 46 of Set 8 | ad

Processing Record 16 of Set 12 | tidore
City not found. Skipping...
Processing Record 17 of Set 12 | yichang
Processing Record 18 of Set 12 | quanzhou
Processing Record 19 of Set 12 | kosovo polje
Processing Record 20 of Set 12 | lavrentiya
Processing Record 21 of Set 12 | waitati
Processing Record 22 of Set 12 | lolua
City not found. Skipping...
Processing Record 23 of Set 12 | itacare
Processing Record 24 of Set 12 | gomel
Processing Record 25 of Set 12 | tieling
Processing Record 26 of Set 12 | narayanpet
Processing Record 27 of Set 12 | tikhvin
Processing Record 28 of Set 12 | panlaitan
Processing Record 29 of Set 12 | kovylkino
Processing Record 30 of Set 12 | husavik
Processing Record 31 of Set 12 | bosaso
Processing Record 32 of Set 12 | izhmorskiy
Processing Record 33 of Set 12 | manokwari
Processing Record 34 of Set 12 | coalcoman
City not found. Skipping...
Processing Record 35 of Set 12 | grodzisk wielkopolski
Processing Record 36 of Set 12 | mungwi
Processing Record 37 of S

In [19]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Current Description,Humidity,Cloudiness,Wind Speed,Country
0,Ushuaia,-54.8000,-68.3000,50.00,few clouds,66,20,4.61,AR
1,Port-Gentil,-0.7193,8.7815,80.46,overcast clouds,74,100,1.81,GA
2,Kralendijk,12.1500,-68.2667,78.80,broken clouds,78,75,13.80,BQ
3,Rikitea,-23.1203,-134.9692,80.11,light rain,73,8,11.52,PF
4,Broome,42.2506,-75.8330,51.80,clear sky,24,1,6.91,US
5,San-Pedro,4.7485,-6.6363,76.39,broken clouds,92,56,6.17,CI
6,Laguna,38.4210,-121.4238,62.60,few clouds,39,20,4.61,US
7,Tshela,-4.9833,12.9333,71.55,overcast clouds,97,100,0.69,CD
8,Port Augusta,-32.5000,137.7667,82.99,clear sky,69,0,11.50,AU
9,Susangerd,31.5608,48.1831,64.40,clear sky,59,0,9.91,IR


In [20]:
new_column_order = ["City", "Country", "Lat",
                    "Lng", "Max Temp", "Humidity",
                    "Cloudiness", "Wind Speed",
                    "Current Description"]
# Assign a new or the same DataFrame the new column order.
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Ushuaia,AR,-54.8000,-68.3000,50.00,66,20,4.61,few clouds
1,Port-Gentil,GA,-0.7193,8.7815,80.46,74,100,1.81,overcast clouds
2,Kralendijk,BQ,12.1500,-68.2667,78.80,78,75,13.80,broken clouds
3,Rikitea,PF,-23.1203,-134.9692,80.11,73,8,11.52,light rain
4,Broome,US,42.2506,-75.8330,51.80,24,1,6.91,clear sky
5,San-Pedro,CI,4.7485,-6.6363,76.39,92,56,6.17,broken clouds
6,Laguna,US,38.4210,-121.4238,62.60,39,20,4.61,few clouds
7,Tshela,CD,-4.9833,12.9333,71.55,97,100,0.69,overcast clouds
8,Port Augusta,AU,-32.5000,137.7667,82.99,69,0,11.50,clear sky
9,Susangerd,IR,31.5608,48.1831,64.40,59,0,9.91,clear sky


In [21]:
#Create the output File (CSV)
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the city_data into a csv
city_data_df.to_csv(output_data_file, index_label="City_ID")